In [6]:
import pandas as pd
import gzip
import json
import re
import os

In [7]:
reviews_path = os.path.join(os.pardir, os.pardir, 'data/reviews.csv')
games_path = os.path.join(os.pardir, os.pardir, 'data/games.csv')

In [8]:
df_reviews = pd.read_csv(reviews_path)
df_games = pd.read_csv(games_path)

In [9]:
df_games.head()

,Unnamed: 0,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",4.49,http://steamcommunity.com/app/761140/reviews/?...,['Single-player'],4.99,False,761140.0,Kotoshiro,NaN,NaN
1,1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free To Play,False,643980.0,Secret Level SRL,Mostly Positive,NaN
2,2,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"['Free to Play', 'Simulation', 'Sports', 'Casu...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"['Single-player', 'Multi-player', 'Online Mult...",Free to Play,False,670290.0,Poolians.com,Mostly Positive,NaN
3,3,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"['Action', 'Adventure', 'Casual']",0.83,http://steamcommunity.com/app/767400/reviews/?...,['Single-player'],0.99,False,767400.0,彼岸领域,NaN,NaN
4,4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"['Action', 'Indie', 'Casual', 'Sports']",1.79,http://steamcommunity.com/app/773570/reviews/?...,"['Single-player', 'Full controller support', '...",2.99,False,773570.0,NaN,NaN,NaN


In [96]:
df_reviews.head()

,Unnamed: 0,username,hours,products,product_id,page_order,date,text,early_access,page,found_funny,compensation,user_id
1,1,₮ʜᴇ Wᴀʀᴛᴏɴ,51.1,769.0,328100,0,2017-12-27,looks like a facebook game,False,1,NaN,NaN,NaN
2,2,hello?<,14.6,2.0,328100,1,2017-10-16,Better than Minecraft,False,1,2.0,Product received for free,NaN
3,3,Cyderine916,5.0,64.0,35140,0,2018-01-04,I love and idolized Batman and this game is Ma...,False,1,NaN,NaN,NaN
4,4,DarklyThinking,16.6,577.0,35140,1,2018-01-04,Still worth playing in 2018.\nProbably my favo...,False,1,NaN,NaN,7.656120e+16
6,6,nd,1.0,2759.0,725280,2,2017-11-28,"endless rooms of some underground complex, blo...",False,1,1.0,NaN,NaN


In [10]:
df_reviews = df_reviews[df_reviews['hours'] >= 1]

In [16]:
df_reviews['freq'] = df_reviews.groupby('product_id')['product_id'].transform('count')

In [17]:
df_reviews.sort_values(by=['freq'], ascending=False, inplace=True)
df_reviews

,Unnamed: 0,username,hours,products,product_id,page_order,date,text,early_access,page,found_funny,compensation,user_id,freq
3897088,3897088,PolarShaman,10.4,22.0,440,8,2015-11-18,An amazing shooter but it is quite hard and it...,False,11235,NaN,NaN,7.656120e+16,179854
3880393,3880393,bleekr,410.4,5.0,440,5,2016-02-19,This game is so fun and addicting,False,9565,NaN,NaN,NaN,179854
3880387,3880387,Decrix,1540.9,190.0,440,9,2016-02-19,i wasted 1540.8 hours on this game then i star...,False,9564,1.0,NaN,NaN,179854
3880388,3880388,Trox,17.4,7.0,440,0,2016-02-19,great m8 i r8 8/8,False,9565,NaN,NaN,7.656120e+16,179854
3880389,3880389,Cpt.KNT,1081.5,35.0,440,1,2016-02-19,Best. Game. Ever!,False,9565,NaN,NaN,NaN,179854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1039473,1039473,LuckyDog,1.4,369.0,567690,0,2017-12-06,Very Odd that there are Zero (0) reviews at th...,False,1,NaN,NaN,7.656120e+16,1
5136181,5136181,Jerrydactyl,3.5,412.0,629420,1,2017-10-07,"It's a nice game, and simply put, if you like ...",False,1,NaN,NaN,NaN,1
3578637,3578637,Blinkdog,3.2,5386.0,661110,0,2017-07-25,this game has promise but has some REALLY bad ...,False,1,NaN,NaN,NaN,1
7365211,7365211,riu525,17.0,1.0,725020,1,2017-11-25,"The game is engaging, very versatile in its co...",False,1,1.0,NaN,7.656120e+16,1


In [18]:
df_reviews[df_reviews['product_id'] == 725280]

,Unnamed: 0,username,hours,products,product_id,page_order,date,text,early_access,page,found_funny,compensation,user_id,freq
6,6,nd,1.0,2759.0,725280,2,2017-11-28,"endless rooms of some underground complex, blo...",False,1,1.0,NaN,NaN,1


In [108]:
df_rev_total['product_id'].unique()[:5]

array([   440, 252490,  49520, 377160, 271590])

In [107]:
top_5 = df_rev_total['product_id'].unique()[:5]
top_5

array([   440, 252490,  49520, 377160, 271590])

In [75]:
top_5_rec = df_games[df_games['id'].isin(top_5)]
print(top_5_rec['app_name'])

25017             Fallout 4
26955    Grand Theft Auto V
29692                  Rust
30746         Borderlands 2
31856       Team Fortress 2
Name: app_name, dtype: object


In [83]:
game0 = top_5_rec.iloc[0]['app_name']
game1 = top_5_rec.iloc[1]['app_name']
game2 = top_5_rec.iloc[2]['app_name']
game3 = top_5_rec.iloc[3]['app_name']
game4 = top_5_rec.iloc[4]['app_name']

In [93]:
def dummy_model(X):
    response = f"""These are the 5 most popular games by review: 
    {game0}
    {game1}
    {game2}
    {game3}
    {game4}"""
    return response

In [94]:
print(dummy_model('my perfect game description'))

These are the 5 most popular games by review: 
    Fallout 4
    Grand Theft Auto V
    Rust
    Borderlands 2
    Team Fortress 2
